In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [8]:
# Gather Data 
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)
features.head()

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])
target.shape

(506, 1)

In [13]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

#property_stats = np.ndarray(shape=(1,11))
#property_stats[0][CRIME_IDX] = features['CRIM'].mean()
#property_stats[0][ZN_IDX] = features['ZN'].mean()
#property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1,11)

array([[3.61352356e+000, 1.13636364e+001, 6.91699605e-002,
        1.48219694e-323, 1.97626258e-323, 2.47032823e-323,
        2.96439388e-323, 3.45845952e-323, 3.95252517e-323,
        4.44659081e-323, 4.94065646e-323]])

In [15]:
type(features.mean())

pandas.core.series.Series

In [16]:
type(features.mean().values)

numpy.ndarray

In [18]:
features.mean().values.reshape(1,11)

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [22]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features) # based on our features dataframe, we are calculating all the predicted values using the thetas from our model.


In [27]:
# Calculating the MSE and RMSE using sklearn
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [54]:
# Creating a Python function that will estimate the log house prices for a specific property
def get_log_estimate(nr_rooms,
                    students_per_classroom,
                    next_to_river=False,
                    high_confidence=True): 
    
    # Configuring our property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1 # If next to the river is true, then we will change one feature(column for Charles River) to 1
    else:
        property_stats[0][CHAS_IDX] = 0 # Otherwise if false, which is the default in this case, property_stats for the Charles River column should be 0.

    # Make Prediction
    
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calculate the range - this is our prediction interval
    if high_confidence:
        # Do X
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
        
    else:
        # Do Y
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [55]:
get_log_estimate(3,20, next_to_river=True, high_confidence=False)

(3.6874009067678504, 3.8749130419649807, 3.49988877157072, 68)

In [56]:
# Calculating the Median house value in Boston in the 1970s
np.median(boston_dataset.target)

21.2

In [63]:
# Python code that converts the log price estimate using 1970s prices

ZILLOW_MEDIAN_PRICE = 583.3 # Based on Zillow's median value
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)
SCALE_FACTOR

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15,
                                              next_to_river=False, high_confidence=False)


# Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar value to nearest thousand
rounded_est = np.around(dollar_est,  -3)
rounded_hi = np.around(dollar_hi,  -3)
rounded_low = np.around(dollar_low,  -3)


print(f'The estimated property value is {rounded_est}')
print(f'At {conf}% confidence the the valuation range is.')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')


The estimated property value is 2055000.0
At 68% the valuation range is
USD 1704000.0 at the lower end to USD 2479000.0 at the high end.


In [78]:
def get_dollar_estimate(rm,ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for the school in the area
    chas -- True if the property is next to the river, False otherwise.
    large_range -- True for a 95% prediction interval, False for a 68% interval.
    """
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm, 
                                                   ptratio,
                                                  next_to_river=chas, 
                                                   high_confidence=large_range)


    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar value to nearest thousand
    rounded_est = np.around(dollar_est,  -3)
    rounded_hi = np.around(dollar_hi,  -3)
    rounded_low = np.around(dollar_low,  -3)


    print(f'The estimated property value is {rounded_est}')
    print(f'At {conf}% confidence the the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')


In [77]:
get_dollar_estimate(rm=6, ptratio=-60, chas=True)

That is unrealistic. Try again.


In [79]:
import boston_valuation as val
val.get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0
At 95% confidence the the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end.
